In [2]:
import pandas as pd
FILE_PATH = "../data/data_parquet"
CONTINENTS = ["asia", "europe", "america"]

In [7]:
df = None
for continent in CONTINENTS:
    subset_df = pd.read_parquet(FILE_PATH+f"/{continent}_data.parquet")
    if continent == CONTINENTS[0]:
        df = subset_df
    else:
        df = pd.concat([df, subset_df], ignore_index=True)


# Ratio: Actions performed by humans and service accounts

In [17]:
user_ratio = df["user_name"].value_counts(normalize=True).to_frame()
user_ratio.to_parquet("../data/visualization/user_ratio.parquet")